# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chui,-33.6971,-53.4616,22.40,54,5,4.34,UY,1705923054
1,1,waitangi,-43.9535,-176.5597,17.68,96,100,6.26,NZ,1705923054
2,2,bilibino,68.0546,166.4372,-20.53,69,100,5.68,RU,1705923054
3,3,georgetown,5.4112,100.3354,27.96,77,20,4.12,MY,1705923055
4,4,avarua,-21.2078,-159.7750,25.03,83,75,2.57,CK,1705923055


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df["Humidity"] < 50) & (city_data_df["Cloudiness"] < 80) & (city_data_df["Max Temp"] > 22) & (city_data_df["Max Temp"] < 30)]

# Drop any rows with null values
cleaned_vacation_df = vacation_df.dropna()

# Display sample data
cleaned_vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,port pirie,-33.1833,138.0167,28.11,31,1,2.83,AU,1705923059
42,42,mangrol,21.1167,70.1167,25.35,47,44,7.11,IN,1705923072
89,89,kukawa,12.9234,13.5606,29.06,16,5,5.34,NG,1705923090
167,167,ad dilam,23.9915,47.1627,25.23,22,0,4.17,SA,1705923115
184,184,broken hill,-31.9500,141.4333,27.17,22,4,3.84,AU,1705923121
241,241,kosum phisai,16.2486,103.0674,28.24,38,27,2.93,TH,1705923141
303,303,mirpur sakro,24.5474,67.6277,24.79,48,43,6.75,PK,1705923167
314,314,justo daract,-33.8594,-65.1828,22.87,48,61,4.35,AR,1705923175
361,361,viedma,-40.8135,-62.9967,28.61,16,10,7.00,AR,1705923183
378,378,nahiyat ash shinafiyah,31.5874,44.6602,25.17,22,32,9.45,IQ,1705923131


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_vacation_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,port pirie,AU,-33.1833,138.0167,31,
42,mangrol,IN,21.1167,70.1167,47,
89,kukawa,NG,12.9234,13.5606,16,
167,ad dilam,SA,23.9915,47.1627,22,
184,broken hill,AU,-31.9500,141.4333,22,
241,kosum phisai,TH,16.2486,103.0674,38,
303,mirpur sakro,PK,24.5474,67.6277,48,
314,justo daract,AR,-33.8594,-65.1828,48,
361,viedma,AR,-40.8135,-62.9967,16,
378,nahiyat ash shinafiyah,IQ,31.5874,44.6602,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port pirie - nearest hotel: Cabin Park
mangrol - nearest hotel: Janta Guest house
kukawa - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
broken hill - nearest hotel: Sturt Motel
kosum phisai - nearest hotel: โรงแรมอิมานรีสอร์ท
mirpur sakro - nearest hotel: No hotel found
justo daract - nearest hotel: No hotel found
viedma - nearest hotel: Níjar
nahiyat ash shinafiyah - nearest hotel: No hotel found
pico truncado - nearest hotel: Pasiones
beaufort west - nearest hotel: Matoppo Inn
lubango - nearest hotel: Novo Hotel
karachi - nearest hotel: Quetta Agha Hotel
luang prabang - nearest hotel: Amantaka


,City,Country,Lat,Lng,Humidity,Hotel Name
13,port pirie,AU,-33.1833,138.0167,31,Cabin Park
42,mangrol,IN,21.1167,70.1167,47,Janta Guest house
89,kukawa,NG,12.9234,13.5606,16,No hotel found
167,ad dilam,SA,23.9915,47.1627,22,No hotel found
184,broken hill,AU,-31.9500,141.4333,22,Sturt Motel
241,kosum phisai,TH,16.2486,103.0674,38,โรงแรมอิมานรีสอร์ท
303,mirpur sakro,PK,24.5474,67.6277,48,No hotel found
314,justo daract,AR,-33.8594,-65.1828,48,No hotel found
361,viedma,AR,-40.8135,-62.9967,16,Níjar
378,nahiyat ash shinafiyah,IQ,31.5874,44.6602,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
city_map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
city_map_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)